In [191]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

In [192]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200806


In [193]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

ディレクトリ

In [194]:
data_dir = Path(r"C:\Users\crie072\Documents\21_Online")

Inputファイル

In [195]:
#セキュポスデータ from CSC
input_file1 = "N029（2020年7月入会）.xlsx"

#デジガレ確認用リスト
input_file2 = "20200731〆【JFRカード御中】成果データ.xlsx"

Outputファイル

In [196]:
output_file1 = f"オンライン獲得レポート集計用データ（通常）_{today_yyyymmdd}"
print(output_file1)

output_file2 = f"オンライン獲得レポート集計用データ（利用条件付き）_{today_yyyymmdd}"
print(output_file2)

オンライン獲得レポート集計用データ（通常）_20200806
オンライン獲得レポート集計用データ（利用条件付き）_20200806


セキュポスデータ取込

In [197]:
df = pd.read_excel(data_dir /r"input"/f"{input_file1}", dtype={'エンボス内容': str})
df['作業日'] = today_yyyymmdd
df.head()

,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD,作業日
0,20200701,1,0,D00,8,31,1,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921910960,46,12.0,7000,40.0,07000,2,0,20200806
1,20200701,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921918211,50,52.0,6000,40.0,06000,2,1,20200806
2,20200701,2,1,D00,8,31,1,0,3,1,1,0,NaN,NaN,NaN,NaN,155051921944480,58,52.0,2000,NaN,02000310413,0,1,20200806
3,20200701,1,0,D02,1,2,1,0,3,102,1,0,NaN,NaN,NaN,NaN,155603602468245,36,12.0,2000,NaN,02000010115,2,0,20200806
4,20200701,5,0,D00,95,0,0,0,3,1,2,0,NaN,NaN,NaN,NaN,155603602537692,47,NaN,0,NaN,NaN,1,0,20200806


In [198]:
#オンライン入会に限定する
df = df[(df["会員カード発行区分"] == "1") 
               & (df["続柄"] == 0) 
               & ~(df["カード提携会社2CD"] == 95) 
               & (df["切替CD"] == 0) 
               & (df["受付部支店"] == 299)]
df.shape[0]

889

In [199]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(889, 25)
Int64Index([   0,    1,    5,   11,   31,   32,   35,   39,   42,   69,
            ...
            6763, 6767, 6772, 6781, 6782, 6787, 6794, 6801, 6832, 6891],
           dtype='int64', length=889)
Index(['カード加入日', '会員カード発行区分', '続柄', 'カード提携会社1CD', 'カード提携会社2CD', 'カード種類CD',
       'カードブランドCD', '切替CD', '勧誘CD', '受付部支店', '性別CD', '再発行CD', '旧カード提携会社1CD',
       '旧カード提携会社2CD', '旧カード種類CD', '旧カードブランドCD', '入会申込番号', '満年齢', 'リボ支払方法CD',
       '勧誘団体CD', '提携カード特殊エリア', 'エンボス内容', 'カード年会費CD', 'リボ利用対象CD', '作業日'],
      dtype='object')
カード加入日           int64
会員カード発行区分       object
続柄               int64
カード提携会社1CD      object
カード提携会社2CD       int64
カード種類CD          int64
カードブランドCD        int64
切替CD             int64
勧誘CD             int64
受付部支店            int64
性別CD             int64
再発行CD            int64
旧カード提携会社1CD     object
旧カード提携会社2CD    float64
旧カード種類CD       float64
旧カードブランドCD     float64
入会申込番号           int64
満年齢              int64
リボ支払方法CD       float64
勧誘団体CD           int64
提携カード特殊

In [200]:
df['カード加入日'] = pd.to_datetime(df['カード加入日'], format = '%Y%m%d')
df['カ－ド加入年'] = df['カード加入日'].dt.year
df['カ－ド加入月'] = df['カード加入日'].dt.month

#"加入年月"のカラム生成
df['カ－ド加入年月'] = df[['カ－ド加入年', 'カ－ド加入月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df.head()

,カード加入日,会員カード発行区分,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,性別CD,再発行CD,旧カード提携会社1CD,旧カード提携会社2CD,旧カード種類CD,旧カードブランドCD,入会申込番号,満年齢,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,リボ利用対象CD,作業日,カ－ド加入年,カ－ド加入月,カ－ド加入年月
0,2020-07-01,1,0,D00,8,31,1,0,3,299,2,0,NaN,NaN,NaN,NaN,155051921910960,46,12.0,7000,40.0,07000,2,0,20200806,2020,7,2020年7月
1,2020-07-01,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921918211,50,52.0,6000,40.0,06000,2,1,20200806,2020,7,2020年7月
5,2020-07-01,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921922544,44,12.0,1000,40.0,01000,2,0,20200806,2020,7,2020年7月
11,2020-07-01,1,0,D00,1,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921940074,33,12.0,2000,40.0,02000,2,0,20200806,2020,7,2020年7月
31,2020-07-01,1,0,D00,8,31,1,0,3,299,1,0,NaN,NaN,NaN,NaN,155051921944332,67,52.0,10000,40.0,10000,2,1,20200806,2020,7,2020年7月


In [201]:
#入会申込番号の重複がないかチェック
print(len(df) == len(df['入会申込番号'].unique()))

True


In [202]:
#重複削除
df = df.drop_duplicates(subset=['入会申込番号'])
df.shape[0]

889

In [203]:
df=df[['入会申込番号','カード加入日','カ－ド加入年月','作業日']]
df.head()

,入会申込番号,カード加入日,カ－ド加入年月,作業日
0,155051921910960,2020-07-01,2020年7月,20200806
1,155051921918211,2020-07-01,2020年7月,20200806
5,155051921922544,2020-07-01,2020年7月,20200806
11,155051921940074,2020-07-01,2020年7月,20200806
31,155051921944332,2020-07-01,2020年7月,20200806


In [204]:
#欠損データがないかチェック
df.isnull().sum()

入会申込番号     0
カード加入日     0
カ－ド加入年月    0
作業日        0
dtype: int64

In [205]:
#df_dup.to_csv(data_dir /r"output"/f"{output_file1}.csv", index=False,header=True,encoding='utf-8-sig')

デジガレのリスト読込

In [224]:
print(input_file2)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file2}", sheet_name = 0)
df_dg = df_dg.rename(columns = {'参照用': '入会申込番号'})

#シート名取得
bk = pd.ExcelFile(data_dir/r"input"/f"{input_file2}")
print(bk.sheet_names)

df_dg.head()

20200731〆【JFRカード御中】成果データ.xlsx
['成果データ', '識別子不明', '成果データ（5月～利用条件追加媒体）', '成果データ（5月～利用条件追加媒体）_承認返却済み']


,ASID,メディア名,媒体チャネル,クリック,申込,申込月,入会申込番号,ASP名,単価,成果,カード発行日
0,49.0,ハピタス（連携用）,ポイント,2020-05-02 00:26:04,2020-05-02 00:58:05,43953,155051921870818,BGT,7000,NaN,NaN
1,619.0,モッピー（連携用）,ポイント,2020-05-02 06:36:10,2020-05-02 06:50:05,43953.3,155051921870917,BGT,7000,NaN,NaN
2,52.0,ECナビSP,ポイント,2020-05-02 08:28:27,2020-05-02 08:35:05,43953.4,155051921870982,BGT,7000,NaN,NaN
3,49.0,ハピタス（連携用）,ポイント,2020-05-02 14:32:46,2020-05-02 15:13:06,43953.6,155051921871626,BGT,7000,NaN,NaN
4,619.0,モッピー（連携用）,ポイント,2020-05-02 15:06:27,2020-05-02 15:24:06,43953.6,155051921871683,BGT,7000,NaN,NaN


In [225]:
print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(617, 11)
RangeIndex(start=0, stop=617, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '申込月', '入会申込番号', 'ASP名', '単価',
       '成果', 'カード発行日'],
      dtype='object')
ASID             float64
メディア名             object
媒体チャネル            object
クリック              object
申込        datetime64[ns]
申込月               object
入会申込番号             int64
ASP名              object
単価                 int64
成果                object
カード発行日            object
dtype: object


In [226]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,入会申込番号,ASP名,単価,成果,カード発行日,申込年月
0,49.0,ハピタス（連携用）,ポイント,2020-05-02 00:26:04,2020-05-02 00:58:05,155051921870818,BGT,7000,NaN,NaN,2020年5月
1,619.0,モッピー（連携用）,ポイント,2020-05-02 06:36:10,2020-05-02 06:50:05,155051921870917,BGT,7000,NaN,NaN,2020年5月
2,52.0,ECナビSP,ポイント,2020-05-02 08:28:27,2020-05-02 08:35:05,155051921870982,BGT,7000,NaN,NaN,2020年5月
3,49.0,ハピタス（連携用）,ポイント,2020-05-02 14:32:46,2020-05-02 15:13:06,155051921871626,BGT,7000,NaN,NaN,2020年5月
4,619.0,モッピー（連携用）,ポイント,2020-05-02 15:06:27,2020-05-02 15:24:06,155051921871683,BGT,7000,NaN,NaN,2020年5月


In [227]:
df_dg.shape[0]

617

In [228]:
pd.crosstab(df_dg['申込年月'],df_dg['媒体チャネル'])

媒体チャネル,ポイント,比較
申込年月,,
2020年1月,3,0
2020年2月,2,0
2020年5月,144,5
2020年6月,147,3
2020年7月,306,6


In [210]:
df_dg = df_dg[['入会申込番号','媒体チャネル','単価','申込年月']]

In [211]:
#2つのテーブルをマージ
df_dg2 = pd.merge(df,df_dg, on =['入会申込番号'], how = 'left')
df_dg2.shape[0]

889

In [212]:
df_dg2.head()

,入会申込番号,カード加入日,カ－ド加入年月,作業日,媒体チャネル,単価,申込年月
0,155051921910960,2020-07-01,2020年7月,20200806,NaN,NaN,NaN
1,155051921918211,2020-07-01,2020年7月,20200806,NaN,NaN,NaN
2,155051921922544,2020-07-01,2020年7月,20200806,NaN,NaN,NaN
3,155051921940074,2020-07-01,2020年7月,20200806,NaN,NaN,NaN
4,155051921944332,2020-07-01,2020年7月,20200806,NaN,NaN,NaN


集計

In [213]:
#客数
df_count = df_dg2.groupby(['作業日','カ－ド加入年月','媒体チャネル','申込年月','単価'])['入会申込番号'].count()
df_count.to_csv(data_dir /r"output"/f"{output_file1}.csv", sep=',',header=True,encoding='utf-8-sig')

df_count.head()

作業日       カ－ド加入年月  媒体チャネル  申込年月     媒体チャネル  単価    
20200806  2020年7月  ポイント    2020年6月  ポイント    7000.0    18
                           2020年7月  ポイント    7000.0    88
Name: 入会申込番号, dtype: int64

利用条件

In [214]:
print(input_file2)

df_dg = pd.read_excel(data_dir/r"input"/f"{input_file2}", sheet_name = 2)
df_dg = df_dg.rename(columns = {'参照用': '入会申込番号'})

#シート名取得
bk = pd.ExcelFile(data_dir/r"input"/f"{input_file2}")
print(bk.sheet_names)

df_dg.head()

20200731〆【JFRカード御中】成果データ.xlsx
['成果データ', '識別子不明', '成果データ（5月～利用条件追加媒体）', '成果データ（5月～利用条件追加媒体）_承認返却済み']


,ASID,メディア名,媒体チャネル,クリック,申込,入会申込番号,ASP名,単価,成果,カード発行日
0,57.0,ライフメディア,ポイント,2020-05-02 01:00:13,2020-05-02 01:17:04,155051921870859,BGT,9000,NaN,NaN
1,37.0,げん玉,ポイント,2020-05-02 07:09:24,2020-05-02 07:21:05,155051921870958,BGT,9000,NaN,NaN
2,57.0,ライフメディア,ポイント,2020-05-02 06:56:25,2020-05-02 07:37:05,155051921870966,BGT,9000,NaN,NaN
3,57.0,ライフメディア,ポイント,2020-05-02 08:36:09,2020-05-02 08:55:04,155051921870990,BGT,9000,NaN,NaN
4,21.0,ポイントインカム,ポイント,2020-05-02 12:03:51,2020-05-02 12:18:04,155051921871345,BGT,9000,NaN,NaN


In [215]:
print(df_dg.shape)
print(df_dg.index)
print(df_dg.columns)
print(df_dg.dtypes)

(793, 10)
RangeIndex(start=0, stop=793, step=1)
Index(['ASID', 'メディア名', '媒体チャネル', 'クリック', '申込', '入会申込番号', 'ASP名', '単価', '成果',
       'カード発行日'],
      dtype='object')
ASID             float64
メディア名             object
媒体チャネル            object
クリック              object
申込        datetime64[ns]
入会申込番号             int64
ASP名              object
単価                 int64
成果                object
カード発行日            object
dtype: object


In [216]:
#"申込年月"のカラム生成
df_dg['申込年'] = df_dg['申込'].dt.year
df_dg['申込月'] = df_dg['申込'].dt.month

df_dg['申込年月'] = df_dg[['申込年', '申込月']].apply(lambda x: '{}年{}月'.format(x[0], x[1]), axis=1)
df_dg = df_dg.drop(columns=['申込年','申込月'])

df_dg.head()

,ASID,メディア名,媒体チャネル,クリック,申込,入会申込番号,ASP名,単価,成果,カード発行日,申込年月
0,57.0,ライフメディア,ポイント,2020-05-02 01:00:13,2020-05-02 01:17:04,155051921870859,BGT,9000,NaN,NaN,2020年5月
1,37.0,げん玉,ポイント,2020-05-02 07:09:24,2020-05-02 07:21:05,155051921870958,BGT,9000,NaN,NaN,2020年5月
2,57.0,ライフメディア,ポイント,2020-05-02 06:56:25,2020-05-02 07:37:05,155051921870966,BGT,9000,NaN,NaN,2020年5月
3,57.0,ライフメディア,ポイント,2020-05-02 08:36:09,2020-05-02 08:55:04,155051921870990,BGT,9000,NaN,NaN,2020年5月
4,21.0,ポイントインカム,ポイント,2020-05-02 12:03:51,2020-05-02 12:18:04,155051921871345,BGT,9000,NaN,NaN,2020年5月


In [217]:
df_dg.shape[0]

793

In [218]:
df_dg = df_dg[['入会申込番号','媒体チャネル','単価','申込年月']]

In [219]:
#2つのテーブルをマージ
df_dg2 = pd.merge(df,df_dg, on =['入会申込番号'], how = 'left')
df_dg2.shape[0]

889

In [220]:
df_dg2.head()

,入会申込番号,カード加入日,カ－ド加入年月,作業日,媒体チャネル,単価,申込年月
0,155051921910960,2020-07-01,2020年7月,20200806,NaN,NaN,NaN
1,155051921918211,2020-07-01,2020年7月,20200806,ポイント,9000.0,2020年6月
2,155051921922544,2020-07-01,2020年7月,20200806,NaN,NaN,NaN
3,155051921940074,2020-07-01,2020年7月,20200806,ポイント,9000.0,2020年6月
4,155051921944332,2020-07-01,2020年7月,20200806,ポイント,9000.0,2020年7月


In [221]:
#客数
df_count = df_dg2.groupby(['作業日','カ－ド加入年月','媒体チャネル','申込年月','単価'])['入会申込番号'].count()
df_count.to_csv(data_dir /r"output"/f"{output_file2}.csv", header=True,encoding='utf-8-sig')

df_count.head()

作業日       カ－ド加入年月  媒体チャネル  申込年月     媒体チャネル  単価    
20200806  2020年7月  ポイント    2020年5月  ポイント    9000.0      2
                           2020年6月  ポイント    9000.0     52
                           2020年7月  ポイント    9000.0    301
Name: 入会申込番号, dtype: int64